In [1]:
import pandas as pd

df = pd.read_csv('data/processed_diabetes_data.csv').drop('Unnamed: 0', axis=1)
df

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,5,1,41,0,1,0,0,0,1,0,...,0,0,1,1,0,0,0,0,1,0
1,15,3,59,0,18,0,0,0,9,0,...,0,0,3,1,0,0,0,0,0,1
2,25,2,11,5,13,2,0,1,6,0,...,0,0,1,1,0,0,0,0,1,1
3,35,2,44,1,16,0,0,0,7,0,...,0,0,3,1,0,0,0,0,0,1
4,45,1,51,0,8,0,0,0,5,0,...,0,0,2,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101758,75,3,51,0,16,0,0,0,9,0,...,0,0,0,1,0,0,0,0,0,1
101759,85,5,33,3,18,0,0,1,9,0,...,0,0,2,1,0,0,0,0,1,1
101760,75,1,53,0,9,1,0,0,13,0,...,0,0,0,1,0,0,0,0,0,1
101761,85,10,45,2,21,0,0,1,9,0,...,0,0,3,1,0,0,0,0,0,1


In [2]:
import dgl
g = dgl.data.from_pandas(df, edge_fmt="src_dst")

node_features = df.to_numpy().values
node_features = torch.from_numpy(node_features).float()

# Create a graph classifier
classifier = dgl.nn.GraphConvClassifier(in_feats=node_features.shape[1], out_feats=num_classes)

# Train the model
optimizer = torch.optim.Adam(classifier.parameters())
for epoch in range(num_epochs):
    logits = classifier(g, node_features)
    loss = F.cross_entropy(logits, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluate the model
preds = classifier(g, node_features).argmax(dim=1)
accuracy = (preds == labels).sum().item() / len(labels)
print("Accuracy:", accuracy)

RuntimeError: DGL requires PyTorch >= 1.12.0

In [ ]:
import torch
import torch.nn as nn
import torch_geometric.nn as gnn

class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GNNModel, self).__init__()

        # Define GNN layers
        self.conv1 = gnn.GCNConv(input_dim, hidden_dim)
        self.conv2 = gnn.GCNConv(hidden_dim, hidden_dim)
        self.conv3 = gnn.GCNConv(hidden_dim, hidden_dim)

        # Output layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply GNN layers
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)

        # Apply output layer
        x = self.fc(x)

        return x

# Define and preprocess your data as a PyTorch Geometric Data object
# Construct the adjacency matrix based on relationships between patients
# Train and evaluate the GNN model on your specific task
